<a href="https://colab.research.google.com/github/Sreya0/BIRD-SPECIES-IDENTIFICATION/blob/main/Bird_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import style

In [ ]:
X = pd.read_csv('data/feature_martix.csv')
Y = pd.read_csv('data/label_matrix.csv')

In [ ]:
main_data = pd.concat([X, Y], axis=1)
main_data = main_data.reindex(np.random.permutation(X.index))
dfs = np.split(main_data, [312], axis=1)

In [ ]:
X = dfs[0]
Y = dfs[1]


In [ ]:
a = Y.values.reshape(11787)

In [ ]:
neww = tf.one_hot(a,200)

In [ ]:
sess = tf.Session()
Y = neww.eval(session=sess)

In [ ]:
batch_size = 100
lr = 0.0002
num_iters = 3300
K = 250
#L = 250
n = 11787
pkeep = tf.placeholder(tf.float32)

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 312])
Y_ = tf.placeholder(tf.float32, shape=[None, 200])

In [ ]:
w1 = tf.Variable(tf.truncated_normal([312, K], stddev=0.1))
b1 = tf.Variable(tf.ones([K]))

In [ ]:
w2 = tf.Variable(tf.truncated_normal([K, 200], stddev=0.1))
b2 = tf.Variable(tf.ones([200]))

In [ ]:
y1 = tf.nn.relu(tf.matmul(x, w1) + b1)
do1 = tf.nn.dropout(y1, pkeep)

In [ ]:
y = tf.nn.softmax(tf.matmul(do1, w2) + b2)

In [ ]:
cross_entropy = -tf.reduce_sum(Y_ * tf.log(y))

In [ ]:
is_correct = tf.equal(tf.argmax(Y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [ ]:
train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(cross_entropy)

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
aaa = []
mmm = []
ccc = []
att = []
testccc = []

In [ ]:
for i in range(num_iters):
    train_data = {x: X[0:8520], Y_: Y[0:8520], pkeep:0.75}

    sess.run(train_step, feed_dict=train_data)

    atr, c = sess.run([accuracy, cross_entropy], feed_dict=train_data)

    test_data = {x: X[7520:], Y_: Y[7520:], pkeep:1}
    at, ac = sess.run([accuracy, cross_entropy], feed_dict=test_data)

    aaa.append(atr)
    mmm.append(i)
    ccc.append(c)
    att.append(at)
    testccc.append(ac)

    if(i%100 == 0):
      print(i, atr, c)

print('Accuracy on training set is ', atr)

print('Accuracy on test set is', at)

In [ ]:
style.use('ggplot')
plt.plot(mmm,aaa,color='b',label='training')
plt.plot(mmm,att,color='r',label='testing')
plt.xlabel('Num Iters')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.plot(mmm,ccc,color='b',label='training')
plt.plot(mmm, testccc,color='r', label='testing')
plt.xlabel('Num Iters')
plt.ylabel('Cross-Entropy')
plt.legend()
plt.show()